In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pickle
import math

In [2]:
import csv

def writeCSV(fname, data):
    write_dir = f"{os.getcwd()}/train-val-test/{save_dir}{fname}.csv"
        
    with open(write_dir, "w") as f:
        # create the csv writer
        writer = csv.writer(f)

        writer.writerow(data)


In [53]:
db = "16.4"
num_windows = "10"
lag = "0"
features = "polar"
channeled = "-channelled"
save_dir = f"db{db}/window-size-{num_windows}/lag{lag}/{features}{channeled}-features/"

In [54]:
if features == "all":
    size = 92
elif features == "polar":
    size = 76
elif features == "ecg":
    size = 16
elif features == "acc":
    size = 63
elif features == "coremotion":
    size = 19

In [55]:
actual_size = size - 3

full_size = math.ceil(math.sqrt(actual_size*int(num_windows) + 2))
dummy = full_size**2 - (actual_size*int(num_windows) + 2)
print(full_size)
print(dummy)

28
52


In [56]:
def savePickle(data, filename):
    with open(os.getcwd() + "/train-val-test/" + filename, "wb") as f: np.save(f, data)

In [57]:
class User:
    def __init__(self, json_doc):
        self.subject_id = json_doc["subject_id"]
        self.name = json_doc["name"]
        self.yob = json_doc["yob"]
        self.height = json_doc["height"]
        self.weight = json_doc["weight"]
        self.is_female = json_doc["is_female"]
        self.medical_conditions = json_doc["medical_conditions"]

In [58]:
if os.path.exists(os.getcwd() + f"/pickles/db{db}/users.p"):
    users = pickle.load(open(os.getcwd() + f"/pickles/db{db}/users.p", "rb"))

In [59]:
class Recording:
    def __init__(self, meta):
        self._id = meta._id
        self.subject_id = meta.subject_id
        self.phone_placement = meta.phone_placement
        self.recording_duration = meta.recording_duration
        self.chunk_ids = meta.chunk_ids
        self.labels = []
        self.p_ecg = []
        self.p_hr = []
        self.p_contact = []
        self.p_acc_x = []
        self.p_acc_y = []
        self.p_acc_z = []
        self.acc_x = []
        self.acc_y = []
        self.acc_z = []
        self.gyr_x = []
        self.gyr_y = []
        self.gyr_z = []
        self.gra_x = []
        self.gra_y = []
        self.gra_z = []
        self.mag_x = []
        self.mag_y = []
        self.mag_z = []
        self.att_roll = []
        self.att_pitch = []
        self.att_yaw = []
        self.delta_heading = []
        self.height = users[self.subject_id].height
        self.weight = users[self.subject_id].weight
        
        print(self._id)
        r_chunks = rec_chunks[self._id]
        for c in r_chunks:
            print(c._id)
        print()
        ordered_chunk_ids = {}
        
        for chunk in r_chunks:
            ordered_chunk_ids[chunk.chunk_index] = chunk
            
        for i in range(1,len(r_chunks)-1):
            chunk = chunks[ordered_chunk_ids[i]._id]
            
            assert(i == chunk.chunk_index)
            
            self.labels += chunk.labels
            self.p_ecg += chunk.p_ecg
            self.p_hr += chunk.p_hr
            self.p_contact += chunk.p_contact
            self.p_acc_x += chunk.p_acc_x
            self.p_acc_y += chunk.p_acc_y
            self.p_acc_z += chunk.p_acc_z
            self.acc_x += chunk.acc_x
            self.acc_y += chunk.acc_y
            self.acc_z += chunk.acc_z
            self.gyr_x += chunk.gyr_x
            self.gyr_y += chunk.gyr_y
            self.gyr_z += chunk.gyr_z
            self.gra_x += chunk.gra_x
            self.gra_y += chunk.gra_y
            self.gra_z += chunk.gra_z
            self.mag_x += chunk.mag_x
            self.mag_y += chunk.mag_y
            self.mag_z += chunk.mag_z
            self.att_roll += chunk.att_roll
            self.att_pitch += chunk.att_pitch
            self.att_yaw += chunk.att_yaw
            self.delta_heading += chunk.delta_heading
            
    def getIntervals(self, lag):
        output = np.zeros((len(self.labels), size))
        returnIdx = -1
        
        for i in range(len(self.labels)):
            data = IntervalData(self, i).flatten()
            if data.shape[0] == size:
                output[i,:] = data
            else:
                returnIdx = i
                break
                
        if lag > 0:
            if returnIdx != -1:
                new_output = output[:returnIdx,:]

                return new_output[:-lag,:-1], new_output[lag:,-1]

            else:
                return output[:-lag,:-1], output[lag:,-1]
        
        elif lag < 0:
            return output[-lag:,:-1], output[:lag,-1]
            
        else:
            return output[:,:-1], output[:,-1]

In [60]:
class IntervalData:
    def __init__(self, rec, idx):
        self.label = [rec.labels[idx]]
        self.p_ecg = rec.p_ecg[idx*13 : idx*13 + 13]
        self.p_acc_x = rec.p_acc_x[idx*20 : idx*20 + 20]
        self.p_acc_y = rec.p_acc_y[idx*20 : idx*20 + 20]
        self.p_acc_z = rec.p_acc_z[idx*20 : idx*20 + 20]
        self.height = rec.height
        self.weight = rec.weight
        if idx < len(rec.acc_x):
            self.acc_x = [rec.acc_x[idx]]
            self.acc_y = [rec.acc_y[idx]]
            self.acc_z = [rec.acc_z[idx]]
            self.gyr_x = [rec.gyr_x[idx]]
            self.gyr_y = [rec.gyr_y[idx]]
            self.gyr_z = [rec.gyr_z[idx]]
            self.gra_x = [rec.gra_x[idx]]
            self.gra_y = [rec.gra_y[idx]]
            self.gra_z = [rec.gra_z[idx]]
            self.mag_x = [rec.mag_x[idx]]
            self.mag_y = [rec.mag_y[idx]]
            self.mag_z = [rec.mag_z[idx]]
            self.att_roll = [rec.att_roll[idx]]
            self.att_pitch = [rec.att_pitch[idx]]
            self.att_yaw = [rec.att_yaw[idx]]
            self.delta_heading = [rec.delta_heading[idx]]
        else:
            self.acc_x = []
            self.acc_y = []
            self.acc_z = []
            self.gyr_x = []
            self.gyr_y = []
            self.gyr_z = []
            self.gra_x = []
            self.gra_y = []
            self.gra_z = []
            self.mag_x = []
            self.mag_y = []
            self.mag_z = []
            self.att_roll = []
            self.att_pitch = []
            self.att_yaw = []
            self.delta_heading = []
        
    def flatten(self):
        #size = 17*1 + 13*1 + 20*3
        output = np.zeros((size))
        print(f"size:{size}")
        
        assert len(self.label) == 1
        
        if features == "ecg":
            if len(self.p_ecg) == 13:
                output[0:13] = np.array(self.p_ecg)
                output[13] = self.height
                output[14] = self.weight
                output[15] = self.label[0]
            else:
                return np.zeros((0))
            
        elif features == "acc":
            if len(self.p_acc_x) == 20:
                output[0:20] = np.array(self.p_acc_x)
                output[20:40] = np.array(self.p_acc_y)
                output[40:60] = np.array(self.p_acc_z)
                output[60] = self.height
                output[61] = self.weight
                output[62] = self.label[0]
            else:
                return np.zeros((0))
            
        elif features == "polar":
            if len(self.p_ecg) == 13 and len(self.p_acc_x) == 20:
                output[0:13] = np.array(self.p_ecg)
                output[13:33] = np.array(self.p_acc_x)
                output[33:53] = np.array(self.p_acc_y)
                output[53:73] = np.array(self.p_acc_z)
                output[73] = self.height
                output[74] = self.weight
                output[75] = self.label[0]
            else:
                return np.zeros((0))
            
        elif features == "coremotion":
            if len(self.acc_x) == 1:
                output[0] = self.acc_x[0]
                output[1] = self.acc_y[0]
                output[2] = self.acc_z[0]
                output[3] = self.gyr_x[0]
                output[4] = self.gyr_y[0]
                output[5] = self.gyr_z[0]
                output[6] = self.gra_x[0]
                output[7] = self.gra_y[0]
                output[8] = self.gra_z[0]
                output[9] = self.mag_x[0]
                output[10] = self.mag_y[0]
                output[11] = self.mag_z[0]
                output[12] = self.att_roll[0]
                output[13] = self.att_pitch[0]
                output[14] = self.att_yaw[0]
                output[15] = self.delta_heading[0]
                output[16] = self.height
                output[17] = self.weight
                output[18] = self.label[0]
            else:
                return np.zeros((0))
                
        elif features == "all":
            if len(self.p_ecg) == 13 and len(self.p_acc_x) == 20 and len(self.acc_x) == 1:
                output[0:13] = np.array(self.p_ecg)
                output[13:33] = np.array(self.p_acc_x)
                output[33:53] = np.array(self.p_acc_y)
                output[53:73] = np.array(self.p_acc_z)
                output[73] = self.acc_x[0]
                output[74] = self.acc_y[0]
                output[75] = self.acc_z[0]
                output[76] = self.gyr_x[0]
                output[77] = self.gyr_y[0]
                output[78] = self.gyr_z[0]
                output[79] = self.gra_x[0]
                output[80] = self.gra_y[0]
                output[81] = self.gra_z[0]
                output[82] = self.mag_x[0]
                output[83] = self.mag_y[0]
                output[84] = self.mag_z[0]
                output[85] = self.att_roll[0]
                output[86] = self.att_pitch[0]
                output[87] = self.att_yaw[0]
                output[88] = self.delta_heading[0]
                output[89] = self.height
                output[90] = self.weight
                output[91] = self.label[0]
            else:
                return np.zeros((0))
        
        return output
        

# Read Pickle Data

In [61]:
ddir = os.getcwd() + f"/pickles/db{db}"
files = os.listdir(ddir)
recordings = []

for f in files:
    if not "users.p" in f:
        reader = open(ddir + "/" + f, "rb")
        rec = pickle.load(reader)
        print(rec._id)
        recordings.append(rec)

AEA56528-8D55-49B1-B0DC-A5CCB861850E
894DB681-F91D-4568-A027-EA97D234F274
A369816A-C3DB-4CD3-984F-01A99AE7CFEC
13F86895-A1D9-42AE-B8B3-E6DF77766A92
00160B9F-7725-41D0-BF42-40679D1FF8C0
1909DCBF-C4B6-4535-9996-5519C03C1C7A
AF7827EE-B69F-4DB6-B670-11A90A24F779
E4ED61E8-E9DD-41F8-9E15-F9054C0DAE84
289789FE-31FC-4B84-A700-5BC0C29CD5FF
8A807260-A8E9-4A8C-AA91-4199C8FAFA55
2A79FBA5-207B-4362-9185-B26DC99004AC
A43647D0-72C1-4547-89B5-AE7198EBD909
71607D86-5C55-4461-9552-F5620D1D7C69
4C9224B7-D6EE-404A-B700-7EC2F471C0E0
5AC7E1D9-8773-4021-B3B7-5216283DC9F1
D05D001A-714C-42FC-825E-EDC90811B8D5
D47BF108-145A-48CE-B05C-A8D57F539CD3
08F36872-722B-4140-8CB8-ACA17C34358E
B6B2C985-1C0A-4FC7-9E8C-E2F05887E57B
610B4316-B72C-4577-88D9-3AF5A538BB3C
71B74433-3A9B-477C-8D07-F5ACB38AD4AB
3B1567A4-5750-4AAA-B9D5-71C8C6674B24
2651A6B5-A53F-421B-83C6-FADC25FC52A8
4D5F47D2-E7D2-4D15-8081-F7BAE8BBCC94
DC15019A-2F4D-4583-B886-1C9ABC01E40A
24E5BE1A-F04C-40B3-8ADB-AF8922DADD7A
CC0E3719-22B8-4008-ABEF-33E17E38B071
C

# Parse Data Into Intervals

In [62]:
intvls = []
for r in recordings:
    print(r._id)
    intvls.append(r.getIntervals(int(lag)))

AEA56528-8D55-49B1-B0DC-A5CCB861850E
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
siz

size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


...

size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76
size:76


# Shift and scale X data

In [59]:
x_mu = np.zeros((size))
rows = 0

for i, xy in enumerate(intvls):
    x_mu += np.sum(xy[0], axis=0)
    rows += xy[0].shape[0]
    
x_mu = x_mu/rows

ValueError: operands could not be broadcast together with shapes (63,) (62,) (63,) 

In [ ]:
x_mu = np.zeros((size))
x_std = np.zeros((size))
rows = 0

for i, xy in enumerate(intvls):
    x_mu += np.sum(xy[0], axis=0)
    x_std += np.std(xy[0], axis=0)
    rows += xy[0].shape[0]
    
x_mu = x_mu/rows
x_std = x_std/rows

In [64]:
x_min = np.zeros((size-1)) + 10000
x_max = np.zeros((size-1))

for i, xy in enumerate(intvls):
    print(xy[0].shape)
    xmin = np.min(xy[0], axis=0)
    for j, x in enumerate(xmin):
        if x < x_min[j]:
            x_min[j] = x
    
    xmax = np.max(xy[0], axis=0)
    for k, x in enumerate(xmax):
        if x > x_max[k]:
            x_max[k] = x

print(x_min.shape)
savePickle(x_min, f"{save_dir}x_min.npy")
savePickle(x_max, f"{save_dir}x_max.npy")
writeCSV("x_min", x_min)
writeCSV("x_max", x_max)
writeCSV("x_max-x_min", x_max - x_min)

(700, 75)
(1600, 75)
(1300, 75)
(1550, 75)
(1150, 75)
(400, 75)
(900, 75)
(1800, 75)
(2300, 75)
(700, 75)
(1100, 75)
(900, 75)
(700, 75)
(1400, 75)
(950, 75)
(900, 75)
(400, 75)
(400, 75)
(1350, 75)
(1350, 75)
(1200, 75)
(1350, 75)
(900, 75)
(1950, 75)
(1350, 75)
(1000, 75)
(1200, 75)
(1500, 75)
(1200, 75)
(1300, 75)
(1350, 75)
(1250, 75)
(2000, 75)
(1000, 75)
(2100, 75)
(1300, 75)
(1300, 75)
(1900, 75)
(1350, 75)
(1500, 75)
(1650, 75)
(1600, 75)
(1400, 75)
(550, 75)
(1500, 75)
(1300, 75)
(1100, 75)
(1150, 75)
(500, 75)
(1050, 75)
(1600, 75)
(75,)


# FFT for signal processing

In [38]:
from scipy.fft import fft, fftfreq

fft_intvls  = []

for i, xy in enumerate(intvls):
    x = xy[0]
    y = xy[1]
    #y = xy[-1]
    #usr_ftrs = xy[-3:-1]
    #x = xy[:-3]
    print(x.shape)
    print(y.shape)
"""
freq = 200
data = acc_z
#fft_data = fft(ecgs)
yf = fft(data)
xf = fftfreq(freq, 1 / freq)

fig, axs = plt.subplots(2)

axs[0].plot(np.arange(len(data)), data)
axs[1].plot(xf, yf)
"""

(700, 75)
(700,)
(1600, 75)
(1600,)
(1300, 75)
(1300,)
(1550, 75)
(1550,)
(1150, 75)
(1150,)
(400, 75)
(400,)
(900, 75)
(900,)
(1800, 75)
(1800,)
(2300, 75)
(2300,)
(700, 75)
(700,)
(1100, 75)
(1100,)
(900, 75)
(900,)
(700, 75)
(700,)
(1400, 75)
(1400,)
(950, 75)
(950,)
(900, 75)
(900,)
(400, 75)
(400,)
(400, 75)
(400,)
(1350, 75)
(1350,)
(1350, 75)
(1350,)
(1200, 75)
(1200,)
(1350, 75)
(1350,)
(900, 75)
(900,)
(1950, 75)
(1950,)
(1350, 75)
(1350,)
(1000, 75)
(1000,)
(1200, 75)
(1200,)
(1500, 75)
(1500,)
(1200, 75)
(1200,)
(1300, 75)
(1300,)
(1350, 75)
(1350,)
(1250, 75)
(1250,)
(2000, 75)
(2000,)
(1000, 75)
(1000,)
(2100, 75)
(2100,)
(1300, 75)
(1300,)
(1300, 75)
(1300,)
(1900, 75)
(1900,)
(1350, 75)
(1350,)
(1500, 75)
(1500,)
(1650, 75)
(1650,)
(1600, 75)
(1600,)
(1400, 75)
(1400,)
(550, 75)
(550,)
(1500, 75)
(1500,)
(1300, 75)
(1300,)
(1100, 75)
(1100,)
(1150, 75)
(1150,)
(500, 75)
(500,)
(1050, 75)
(1050,)
(1600, 75)
(1600,)


'\nfreq = 200\ndata = acc_z\n#fft_data = fft(ecgs)\nyf = fft(data)\nxf = fftfreq(freq, 1 / freq)\n\nfig, axs = plt.subplots(2)\n\naxs[0].plot(np.arange(len(data)), data)\naxs[1].plot(xf, yf)\n'

In [37]:
a = np.zeros((2, 2))
a[0, :-2] = [1]*2
a

ValueError: could not broadcast input array from shape (2,) into shape (0,)

# Sliding Windows

In [65]:
x_max - x_min

array([39463., 39463., 39463., 39463., 39463., 39463., 39463., 39463.,
       39463., 39463., 39463., 39463., 39463., 15033., 15643., 15678.,
       15569., 15802., 14049., 15140., 15379., 15863., 13660., 14994.,
       15904., 15504., 13077., 14826., 15860., 13174., 13049., 14003.,
       14250., 14471., 15548., 15247., 14794., 15921., 15506., 14591.,
       15682., 15947., 15930., 15984., 15984., 15283., 15894., 14889.,
       14028., 15293., 15932., 15039., 15359., 14973., 13526., 13281.,
       15467., 14741., 13572., 13462., 14275., 15146., 14791., 15253.,
       15791., 15968., 15222., 14889., 14997., 15397., 15429., 15695.,
       14902.,  1575.,    94.])

In [66]:
import more_itertools as mit

def window(seq, n=10):
    return np.array([list(mit.flatten(s)) for s in mit.windowed(seq, n)])

def window_1d(seq, n=10):
    return np.lib.stride_tricks.sliding_window_view(seq, window_shape = n)

## Window channels with averaged labels

In [67]:
ys = []
ys_w = []
xs = []
num_windows = 10

for x,y in intvls:
    norm_data = (x - x_min)/(x_max - x_min)
    x_sensor = norm_data[:,:-2]
    x_user = norm_data[0,-2:]
    
    xdata = []
    for row in range(x_sensor.shape[0]):
        xdata.append(x_sensor[row,:])
    
    wind_x = list(mit.windowed(xdata, num_windows))
    
    all_new_x = np.zeros((len(wind_x), num_windows, x_sensor.shape[1] + 2))
    
    for i, window in enumerate(wind_x):
        for idx, intvl in enumerate(window):
            all_new_x[i, idx, :73] = intvl
            all_new_x[i, idx, -2:] = x_user

    xs.append(all_new_x)
    
    ys_w.append(window_1d(y, num_windows))
    ys.append(np.mean(ys_w[-1], axis=1))
    
    print(xs[-1].shape)
    print(ys_w[-1].shape)
    print(ys[-1].shape)
    print()

(691, 10, 75)
(691, 10)
(691,)

(1591, 10, 75)
(1591, 10)
(1591,)

(1291, 10, 75)
(1291, 10)
(1291,)

(1541, 10, 75)
(1541, 10)
(1541,)

(1141, 10, 75)
(1141, 10)
(1141,)

(391, 10, 75)
(391, 10)
(391,)

(891, 10, 75)
(891, 10)
(891,)

(1791, 10, 75)
(1791, 10)
(1791,)

(2291, 10, 75)
(2291, 10)
(2291,)

(691, 10, 75)
(691, 10)
(691,)

(1091, 10, 75)
(1091, 10)
(1091,)

(891, 10, 75)
(891, 10)
(891,)

(691, 10, 75)
(691, 10)
(691,)

(1391, 10, 75)
(1391, 10)
(1391,)

(941, 10, 75)
(941, 10)
(941,)

(891, 10, 75)
(891, 10)
(891,)

(391, 10, 75)
(391, 10)
(391,)

(391, 10, 75)
(391, 10)
(391,)

(1341, 10, 75)
(1341, 10)
(1341,)

(1341, 10, 75)
(1341, 10)
(1341,)

(1191, 10, 75)
(1191, 10)
(1191,)

(1341, 10, 75)
(1341, 10)
(1341,)

(891, 10, 75)
(891, 10)
(891,)

(1941, 10, 75)
(1941, 10)
(1941,)

(1341, 10, 75)
(1341, 10)
(1341,)

(991, 10, 75)
(991, 10)
(991,)

(1191, 10, 75)
(1191, 10)
(1191,)

(1491, 10, 75)
(1491, 10)
(1491,)

(1191, 10, 75)
(1191, 10)
(1191,)

(1291, 10, 75)
(1291,

## Dummy features for CNN

In [31]:
ys = []
ys_w = []
xs = []
num_windows = 10

for x,y in intvls:
    norm_data = (x - x_min)/(x_max - x_min)
    x_sensor = norm_data[:,:-2]
    x_user = norm_data[0,-2:]
    
    xdata = []
    for row in range(x_sensor.shape[0]):
        xdata.append(x_sensor[row,:])
    
    wind_x = list(mit.windowed(xdata, num_windows))
    
    all_new_x = np.zeros((len(wind_x), num_windows*x_sensor.shape[1] + 2))
    all_new_x[:,-2:] = x_user
    
    for i,window in enumerate(wind_x):
        all_new_x[i,:-2] = [ftr for intvl in window for ftr in intvl]
    
    #final_x = np.zeros((all_new_x.shape[0], all_new_x.shape[1] + dummy))
    #final_x[:,:-dummy] = all_new_x
    
    final_x = np.zeros((all_new_x.shape[0], all_new_x.shape[1]))
    final_x[:,:] = all_new_x
    
    xs.append(final_x)
    
    ys_w.append(window_1d(y, num_windows))
    ys.append(ys_w[-1][:,0])
    
    print(xs[-1].shape)
    print(ys_w[-1].shape)
    print(ys[-1].shape)
    print()

(691, 732)
(691, 10)
(691,)

(1591, 732)
(1591, 10)
(1591,)

(1291, 732)
(1291, 10)
(1291,)

(1541, 732)
(1541, 10)
(1541,)

(1141, 732)
(1141, 10)
(1141,)

(391, 732)
(391, 10)
(391,)

(891, 732)
(891, 10)
(891,)

(1791, 732)
(1791, 10)
(1791,)

(2291, 732)
(2291, 10)
(2291,)

(691, 732)
(691, 10)
(691,)

(1091, 732)
(1091, 10)
(1091,)

(891, 732)
(891, 10)
(891,)

(691, 732)
(691, 10)
(691,)

(1391, 732)
(1391, 10)
(1391,)

(941, 732)
(941, 10)
(941,)

(891, 732)
(891, 10)
(891,)

(391, 732)
(391, 10)
(391,)

(391, 732)
(391, 10)
(391,)

(1341, 732)
(1341, 10)
(1341,)

(1341, 732)
(1341, 10)
(1341,)

(1191, 732)
(1191, 10)
(1191,)

(1341, 732)
(1341, 10)
(1341,)

(891, 732)
(891, 10)
(891,)

(1941, 732)
(1941, 10)
(1941,)

(1341, 732)
(1341, 10)
(1341,)

(991, 732)
(991, 10)
(991,)

(1191, 732)
(1191, 10)
(1191,)

(1491, 732)
(1491, 10)
(1491,)

(1191, 732)
(1191, 10)
(1191,)

(1291, 732)
(1291, 10)
(1291,)

(1341, 732)
(1341, 10)
(1341,)

(1241, 732)
(1241, 10)
(1241,)

(1991, 732)


# Merge windows

In [49]:
num_rows = np.sum([x.shape[0] for x in xs])
print(num_rows)

62791


## Window channels with averaged labels

In [51]:
xx = np.zeros((num_rows, num_windows, size-1))
yy = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys[i].shape[0])
    xx[offset : offset+xs[i].shape[0], :, :] = xs[i]
    yy[offset : offset+xs[i].shape[0]] = ys[i]
    offset += xs[i].shape[0]

## Dummy features for CNN

In [19]:
xx = np.zeros((num_rows, num_windows*(size-3)+2+dummy))
yy = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys[i].shape[0])
    
    xx[offset : offset+xs[i].shape[0], :] = xs[i]
    yy[offset : offset+xs[i].shape[0]] = ys[i]
    offset += xs[i].shape[0]

In [20]:
shaped_xx = np.zeros((xx.shape[0], 1, full_size, full_size))
for row in range(xx.shape[0]):
    shaped_xx[row, :, :, :] = xx[row,:].reshape((1,1,full_size, full_size))
print(shaped_xx.shape)

(62791, 1, 12, 12)


In [21]:
savePickle(xx, f"{save_dir}xx.npy")
savePickle(shaped_xx, f"{save_dir}shaped_xx.npy")
savePickle(yy, f"{save_dir}yy.npy")

In [64]:
yy_re = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys_w[i].shape[0])
    
    for j in range(xs[i].shape[0]):
        yy_re[offset+j] = np.sum([(2*ys_w[i][j,k])**(num_windows-k) for k in range(num_windows)])
        
    offset += xs[i].shape[0]
    
yy_re = yy_re - np.mean(yy_re)
yy_re = yy_re / np.std(yy_re)

In [65]:
yy_rs = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys_w[i].shape[0])
    yy_rs[offset : offset+xs[i].shape[0]] = np.sum(ys_w[i], axis=1)
    offset += xs[i].shape[0]
    
yy_rs = yy_rs - np.mean(yy_rs)
yy_rs = yy_rs / np.std(yy_rs)

In [22]:
from sklearn.model_selection import train_test_split

X_tmp, X_test, y_tmp, y_test = train_test_split(xx, yy, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [23]:
savePickle(X_train, f"{save_dir}us-X_train.npy")
savePickle(X_test, f"{save_dir}us-X_test.npy")
savePickle(X_val, f"{save_dir}us-X_val.npy")
savePickle(y_train, f"{save_dir}y_train.npy")
savePickle(y_test, f"{save_dir}y_test.npy")
savePickle(y_val, f"{save_dir}y_val.npy")

In [24]:
from sklearn.model_selection import train_test_split

X_tmp, X_test, y_tmp, y_test = train_test_split(shaped_xx, yy, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [25]:
savePickle(X_train, f"{save_dir}X_train.npy")
savePickle(X_test, f"{save_dir}X_test.npy")
savePickle(X_val, f"{save_dir}X_val.npy")

In [42]:
X_train.shape

(37582, 1, 28, 28)

In [68]:
Xre_tmp, Xre_test, yre_tmp, yre_test = train_test_split(xx, yy_re, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
Xre_train, Xre_val, yre_train, yre_val = train_test_split(Xre_tmp, yre_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [69]:
savePickle(Xre_train, f"{save_dir}Xre_train.npy")
savePickle(Xre_test, f"{save_dir}Xre_test.npy")
savePickle(Xre_val, f"{save_dir}Xre_val.npy")
savePickle(yre_train, f"{save_dir}yre_train.npy")
savePickle(yre_test, f"{save_dir}yre_test.npy")
savePickle(yre_val, f"{save_dir}yre_val.npy")

In [70]:
Xrs_tmp, Xrs_test, yrs_tmp, yrs_test = train_test_split(xx, yy_rs, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
Xrs_train, Xrs_val, yrs_train, yrs_val = train_test_split(Xrs_tmp, yrs_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [71]:
savePickle(Xrs_train, f"{save_dir}Xrs_train.npy")
savePickle(Xrs_test, f"{save_dir}Xrs_test.npy")
savePickle(Xrs_val, f"{save_dir}Xrs_val.npy")
savePickle(yrs_train, f"{save_dir}yrs_train.npy")
savePickle(yrs_test, f"{save_dir}yrs_test.npy")
savePickle(yrs_val, f"{save_dir}yrs_val.npy")

In [32]:
xx.shape

(28952, 910)